## Calendar


In [1]:

from src.data import all_functions as af
import pandas as pd


In [2]:
import requests
import pandas as pd
from src.data import all_functions as af

PLAYER_COLUMN = "PLAYER"
TEAM_KEY = "team_key"
PLAYER_ID_KEY = "player_id"
NAME_KEY = "name"
MEAN_ROW = "mean"
NAME_KEY = "name"
LSCD_KEY = "lscd"
MSCD_KEY = "mscd"
G_KEY = "g"
GID_KEY = "gid"
GAME_ID_KEY = "game_id"
GDATE_KEY = "gdte"
GAME_DATE_KEY= "game_date"
V_KEY = "v"
TID_KEY = "tid"
VISITORS_TEAM_ID_KEY = "visitors_tid"
VISITORS_KEY =  "visitors"
TC_KEY = "tc"
TN_KEY = "tn"
H_KEY = "h"
HOME_TEAM_ID_KEY = "home_tid"
HOME_KEY = "home"
TOTAL_ROW = "total"
INDEX_COLUMN = "index"
NBA_NAME_LA_CLIPPERS = "LA Clippers"
YAHOO_NAME_LA_CLIPPERS = "Los Angeles Clippers"
COLUMN_9CAT_DECIMAL_FORMAT = {af.FIELD_GOAL_PERCENTAGE_COLUMN : af.THREE_DECIMAL_FORMAT,
     af.FREE_THROW_PERCENTAGE_COLUMN : af.THREE_DECIMAL_FORMAT,
     af.THREES_MADE_COLUMN : af.TWO_DECIMAL_FORMAT,
     af.POINTS_COLUMN : af.TWO_DECIMAL_FORMAT,
     af.REBOUNDS_COLUMN : af.TWO_DECIMAL_FORMAT,
     af.ASSITS_COLUMN : af.TWO_DECIMAL_FORMAT, af.STEALS_COLUMN : af.TWO_DECIMAL_FORMAT,
     af.BLOCKS_COLUMN : af.TWO_DECIMAL_FORMAT, af.TURNOVERS_COLUMN : af.TWO_DECIMAL_FORMAT}

def get_game_information_in_month(nba_json):
    """
    Function loops through a month (n) in nba_json and creates a dataframe containing game
    information, game_id, home/visitor team/team_id and game_date for each month.
    param nba_json (dictionary): dictionary containing game information
    returns game_dataframe (pandas dataframe): dataframe containing game information for a month
    """
    game_dataframe = pd.DataFrame()
    for n in range(len(nba_json)):
        game_dictionary = dict()
        game_dictionary[GID_KEY] = nba_json[n][GID_KEY]
        game_dictionary[GDATE_KEY] = nba_json[n][GDATE_KEY]
        game_dictionary[VISITORS_TEAM_ID_KEY] = nba_json[n][V_KEY][TID_KEY]
        game_dictionary[VISITORS_KEY] = nba_json[n][V_KEY][TC_KEY] + " " \
                                        + nba_json[n][V_KEY][TN_KEY]
        game_dictionary[HOME_TEAM_ID_KEY] = nba_json[n][H_KEY][TID_KEY]
        game_dictionary[HOME_KEY] = nba_json[n][H_KEY][TC_KEY] + " " \
                                    + nba_json[n][H_KEY][TN_KEY]
        game_dataframe = game_dataframe.append(game_dictionary, ignore_index=True)
    return game_dataframe

def get_game_information_in_season(season):
    """
    Function calls the NBA endpoint that returns the season schedule. It then loops through each
    month in that schedule to create a dataframe containing game information, game_id,
    home/visitor team/team_id and game_date for the season.
    param season (string): season for schedule
    returns season_games_dataframe (pandas dataframe): dataframe containing game information for
    a season
    """
    r = requests.get(f"http://data.nba"
                     f".com/data/10s/v2015/json/mobile_teams/nba/{season}/league/00_full_schedule"
                     f".json")
    game_information_json = r.json()[LSCD_KEY]
    season_games_dataframe = pd.DataFrame()
    for n in range(len(game_information_json)):
        month_n_game_dataframe = get_game_information_in_month\
            (game_information_json[n][MSCD_KEY][G_KEY])
        season_games_dataframe = season_games_dataframe.append(month_n_game_dataframe)
    return season_games_dataframe

def get_week_current_week_information():
    """
    Function finds the current Yahoo Fantasy week number and its start and end date.
    return current_fantasy_week (int): Yahoo Fantasy week number
    return week_start_date (string): start date of Yahoo Fantasy week number
    return week_end_date (string): end date of Yahoo Fantasy week number
    """
    sc = af.yahoo_fantasy_api_authentication()
    league = af.yahoo_fantasy_league(sc)
    current_fantasy_week = league.current_week()
    week_start_date = league.week_date_range(current_fantasy_week)[0]
    week_end_date = league.week_date_range(current_fantasy_week)[1]
    return current_fantasy_week, week_start_date, week_end_date

def get_next_week_information():
    """
    Function finds the next Yahoo Fantasy week number and its start and end date.
    return next_fantasy_week (int): Yahoo Fantasy week number
    return week_start_date (string): start date of Yahoo Fantasy week number
    return week_end_date (string): end date of Yahoo Fantasy week number
    """
    sc = af.yahoo_fantasy_api_authentication()
    league = af.yahoo_fantasy_league(sc)
    next_fantasy_week = league.current_week() + 1
    week_start_date = league.week_date_range(next_fantasy_week)[0]
    week_end_date = league.week_date_range(next_fantasy_week)[1]
    return next_fantasy_week, week_start_date, week_end_date

def get_player_ids_names_in_fantasy_team(team_dict):
    """
    Function finds the Yahoo Fantasy player ids and player names rostered by a team in the league
    param team_dict (dictionary): dictionary containing information about a owners team
    return player_id_name_team_list (list): Yahoo Fantasy player ids and names
    return player_name_list (list): Yahoo Fantasy player names
    """
    sc = af.yahoo_fantasy_api_authentication()
    league = af.yahoo_fantasy_league(sc)
    team = league.to_team(team_dict[TEAM_KEY])
    team_roster = team.roster()
    player_id_name_team_list = [{PLAYER_ID_KEY : player[PLAYER_ID_KEY], NAME_KEY : player[NAME_KEY]}
                           for player in team_roster]
    player_name_list = [player[NAME_KEY] for player in player_id_name_team_list]
    return player_id_name_team_list, player_name_list

def get_fantasy_week_games_dataframe(season_schedule_dataframe, week_start_date, week_end_date):
    """
    Function filters the entire NBA season schedule on a start/end date to find the scheduled
    games between specific dates. It returns the amount of games each NBA team plays during that
    period of time.
    param season_schedule_dataframe (pandas dataframe): dataframe containing the entire seasons
    NBA schedule
    param week_start_date (string): date to begin filtering by
    param week_start_date (string): date to end filtering by
    return current_fantasy_games_week (pandas dataframe) dataframe containing the scheduled games
    in specified param dates
    return team_game_counts (pandas series): series containing count of games for each NBA team
    """
    filtered_games_week = season_schedule_dataframe[(season_schedule_dataframe[GDATE_KEY] >=
                                                   str(week_start_date)) &
                                                  (season_schedule_dataframe[GDATE_KEY] <=
                                                   str(week_end_date))]
    team_game_counts = filtered_games_week[HOME_KEY].append\
        (filtered_games_week[VISITORS_KEY]).value_counts()
    return filtered_games_week, team_game_counts

def get_player_games(team_game_counts_series, player_name_list):
    """
    Function creates a dictionary containing key/value pairs of player names/games in a series of
    NBA team game counts
    param team_game_counts_series (pandas series): series of team name and games playing
    param player_name_list (list): list of player names
    return player_games_dictionary (dictionary): key/value pairing of player names/games playing
    """
    player_games_dictionary = dict()
    team_game_counts_dataframe = pd.DataFrame(team_game_counts_series)
    team_game_counts_dataframe.reset_index(inplace=True)
    team_game_counts_dataframe = team_game_counts_dataframe.replace(NBA_NAME_LA_CLIPPERS,
                                                                    YAHOO_NAME_LA_CLIPPERS)
    for player in player_name_list:
        team = af.yahoo_player_team_and_jersey(player)[0]
        try:
            player_games_dictionary[player] = \
                team_game_counts_dataframe[team_game_counts_dataframe["index"] == team][0].iloc[0]
        except IndexError:
            player_games_dictionary[player] = 0
    return player_games_dictionary

def get_team_player_game_dataframe(team_dict, week="next", season=2020):
    """

    @param team_dict:
    @param week:
    @param season:
    @return:
    """
    if week == "next":
        fantasy_week, week_start_date, week_end_date = get_next_week_information()
    elif week == "current":
        fantasy_week, week_start_date, week_end_date = get_week_current_week_information()
    else:
        raise ValueError("Parameter 'week' must be 'current' or 'next'.")
    season_games_dataframe = get_game_information_in_season(season) # Run once only
    filtered_games_week, team_game_counts = get_fantasy_week_games_dataframe\
        (season_games_dataframe, week_start_date, week_end_date)
    player_id_name_team_list, player_name_list = get_player_ids_names_in_fantasy_team(team_dict)
    player_games_dictionary = get_player_games(team_game_counts, player_name_list)
    return filtered_games_week, player_games_dictionary

In [4]:
filtered_games_week, player_games_dictionary = get_team_player_game_dataframe(af
                                                                            .NUNN_OF_YALL_BETTA, week="current", season=2020)

[2021-02-22 23:31:02,679 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-02-22 23:31:02,692 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 4902.815487861633
[2021-02-22 23:31:02,693 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN HAS EXPIRED
[2021-02-22 23:31:02,693 DEBUG] [yahoo_oauth.oauth.refresh_access_token] REFRESHING TOKEN
[2021-02-22 23:31:03,513 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-02-22 23:31:03,523 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 0.8294339179992676
[2021-02-22 23:31:03,524 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-02-22 23:31:07,567 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-02-22 23:31:07,571 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 4.877391576766968
[2021-02-22 23:31:07,572 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-02-22 23:31:08,545 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-02-22 23:31:08,548 DEBUG] [yahoo_oauth.oauth.token_is_vali

In [11]:
filtered_games_week

,gdte,gid,home,home_tid,visitors,visitors_tid
168,2021-02-22,0022000473,Indiana Pacers,1.610613e+09,San Antonio Spurs,1.610613e+09
169,2021-02-22,0022000474,Houston Rockets,1.610613e+09,Chicago Bulls,1.610613e+09
170,2021-02-22,0022000476,Dallas Mavericks,1.610613e+09,Memphis Grizzlies,1.610613e+09
171,2021-02-22,0022000475,Oklahoma City Thunder,1.610613e+09,Miami Heat,1.610613e+09
172,2021-02-22,0022000477,Phoenix Suns,1.610613e+09,Portland Trail Blazers,1.610613e+09
173,2021-02-22,0022000478,Utah Jazz,1.610613e+09,Charlotte Hornets,1.610613e+09
174,2021-02-22,0022000479,Los Angeles Lakers,1.610613e+09,Washington Wizards,1.610613e+09
175,2021-02-23,0022000480,Cleveland Cavaliers,1.610613e+09,Atlanta Hawks,1.610613e+09
176,2021-02-23,0022000481,Orlando Magic,1.610613e+09,Detroit Pistons,1.610613e+09
177,2021-02-23,0022000482,Brooklyn Nets,1.610613e+09,Sacramento Kings,1.610613e+09


In [13]:
player_games_dictionary



{'Patty Mills': 3,
 'Gary Trent Jr.': 3,
 'Malcolm Brogdon': 4,
 'Jimmy Butler': 4,
 'Jae Crowder': 4,
 'Duncan Robinson': 4,
 'Wendell Carter Jr.': 4,
 'Enes Kanter': 3,
 'Kyle Kuzma': 4,
 'Cameron Johnson': 4,
 'Marcus Smart': 4,
 'Khris Middleton': 3,
 'Nikola Jokic': 3,
 'Pascal Siakam': 4,
 'Jeremy Lamb': 4}

In [18]:
season_games_dataframe = get_game_information_in_season(2020)
fantasy_week, week_start_date, week_end_date = get_week_current_week_information()
filtered_games_week, team_game_counts = get_fantasy_week_games_dataframe\
    (season_games_dataframe, week_start_date, week_end_date)
player_id_name_team_list, player_name_list = get_player_ids_names_in_fantasy_team(af
                                                                            .NUNN_OF_YALL_BETTA)

[2021-02-22 22:44:53,890 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-02-22 22:44:53,893 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 2134.016390323639
[2021-02-22 22:44:53,894 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-02-22 22:44:55,725 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-02-22 22:44:55,728 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 2135.8518550395966
[2021-02-22 22:44:55,729 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID


In [24]:
filtered_filtered_games_week = filtered_games_week[filtered_games_week["gdte"] == "2021-02-22"]

In [27]:
team_game_counts = filtered_filtered_games_week[HOME_KEY].append\
    (filtered_filtered_games_week[VISITORS_KEY]).value_counts()
team_game_counts

San Antonio Spurs         1
Utah Jazz                 1
Oklahoma City Thunder     1
Miami Heat                1
Portland Trail Blazers    1
Washington Wizards        1
Phoenix Suns              1
Houston Rockets           1
Los Angeles Lakers        1
Indiana Pacers            1
Chicago Bulls             1
Charlotte Hornets         1
Dallas Mavericks          1
Memphis Grizzlies         1
dtype: int64

In [29]:
player_id_name_team_list, player_name_list = get_player_ids_names_in_fantasy_team(af
                                                                            .NUNN_OF_YALL_BETTA)

[2021-02-22 22:49:32,068 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-02-22 22:49:32,071 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 2412.1949574947357
[2021-02-22 22:49:32,072 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID


In [30]:
player_games_dictionary = get_player_games(team_game_counts, player_name_list)

[2021-02-22 22:49:40,767 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-02-22 22:49:40,771 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 2420.8945808410645
[2021-02-22 22:49:40,772 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-02-22 22:49:41,714 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-02-22 22:49:41,716 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 2421.840026140213
[2021-02-22 22:49:41,717 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-02-22 22:49:42,768 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-02-22 22:49:42,772 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 2422.8954632282257
[2021-02-22 22:49:42,773 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-02-22 22:49:43,790 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-02-22 22:49:43,793 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 2423.916601419449
[2021-02-22 22:49:43,794 DEBUG] [yahoo_oauth.oauth

{'Patty Mills': 1,
 'Gary Trent Jr.': 1,
 'Malcolm Brogdon': 1,
 'Jimmy Butler': 1,
 'Jae Crowder': 1,
 'Duncan Robinson': 1,
 'Wendell Carter Jr.': 1,
 'Enes Kanter': 1,
 'Kyle Kuzma': 1,
 'Cameron Johnson': 1,
 'Marcus Smart': 0,
 'Khris Middleton': 0,
 'Nikola Jokic': 0,
 'Pascal Siakam': 0,
 'Jeremy Lamb': 1}

In [35]:
team_name_list = list()
for player in player_name_list:
    team = af.yahoo_player_team_and_jersey(player)[0]
    team_name_list.append(team)

[2021-02-22 23:00:23,445 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-02-22 23:00:23,448 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 3063.57178068161
[2021-02-22 23:00:23,449 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-02-22 23:00:24,343 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-02-22 23:00:24,347 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 3064.4704389572144
[2021-02-22 23:00:24,348 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-02-22 23:00:25,306 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-02-22 23:00:25,309 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 3065.4322288036346
[2021-02-22 23:00:25,310 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-02-22 23:00:26,241 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-02-22 23:00:26,244 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 3066.367441177368
[2021-02-22 23:00:26,245 DEBUG] [yahoo_oauth.oauth.

In [3]:
def get_team_name_list_from_player_name_list(player_name_list):
    """

    @param player_name_list:
    @return:
    """
    team_name_list = list()
    for player in player_name_list:
        team = af.yahoo_player_team_and_jersey(player)[0]
        team_name_list.append(team)
    return player_name_list, team_name_list

def daily_get_player_games(team_game_counts_series, player_name_list, team_name_list):
    """

    @param team_game_counts_series:
    @param player_name_list:
    @param team_name_list:
    @return:
    """
    player_games_dictionary = dict()
    team_game_counts_dataframe = pd.DataFrame(team_game_counts_series)
    team_game_counts_dataframe.reset_index(inplace=True)
    team_game_counts_dataframe = team_game_counts_dataframe.replace(NBA_NAME_LA_CLIPPERS,
                                                                    YAHOO_NAME_LA_CLIPPERS)
    for player, team in zip(player_name_list,team_name_list):
        try:
            player_games_dictionary[player] = \
            team_game_counts_dataframe[team_game_counts_dataframe["index"] == team][0].iloc[0]
        except IndexError:
            player_games_dictionary[player] = 0
    return player_games_dictionary

In [7]:
season_games_dataframe = get_game_information_in_season(2020)
fantasy_week, week_start_date, week_end_date = get_week_current_week_information()
filtered_games_week, team_game_counts = get_fantasy_week_games_dataframe\
    (season_games_dataframe, week_start_date, week_end_date)


player_id_name_team_list, player_name_list = get_player_ids_names_in_fantasy_team(af
                                                                            .NUNN_OF_YALL_BETTA)
player_name_list, team_name_list = get_team_name_list_from_player_name_list(player_name_list)

daily_game_count_list = list()
for day in list(filtered_games_week["gdte"].unique()):
    daily_dataframe = filtered_games_week[filtered_games_week["gdte"] == day]
    daily_team_game_counts = daily_dataframe[HOME_KEY].append(daily_dataframe[VISITORS_KEY])\
        .value_counts()
    player_games_dictionary = daily_get_player_games(daily_team_game_counts, player_name_list,
                                                     team_name_list)
    daily_game_count_list.append(sum(list(player_games_dictionary.values())))
cleaned_daily_game_count_list = [10 if i >= 10 else i for i in daily_game_count_list]

[2021-02-22 23:31:25,325 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-02-22 23:31:25,328 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 22.633565425872803
[2021-02-22 23:31:25,329 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-02-22 23:31:27,173 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-02-22 23:31:27,175 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 24.48088788986206
[2021-02-22 23:31:27,176 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-02-22 23:31:28,204 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-02-22 23:31:28,207 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 25.51259684562683
[2021-02-22 23:31:28,208 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-02-22 23:31:29,117 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-02-22 23:31:29,119 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 26.42459464073181
[2021-02-22 23:31:29,120 DEBUG] [yahoo_oauth.oauth.

In [51]:
daily_game_count_list

[11, 6, 11, 2, 12, 4, 9]

In [10]:
cleaned_daily_game_count_list = [10 if i >= 10 else i for i in daily_game_count_list]
cleaned_daily_game_count_list

[10, 6, 10, 2, 10, 4, 9]

In [16]:
total_and_daily_games_list = [sum(cleaned_daily_game_count_list)] + \
                             cleaned_daily_game_count_list
total_and_daily_games_list

[51, 10, 6, 10, 2, 10, 4, 9]

In [22]:
DAILY_COLUMNS = ["Total", "Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]
matchup_daily_game_dataframe = pd.DataFrame()
matchup_daily_game_dataframe["day"] = total_and_daily_games_list
matchup_daily_game_dataframe = matchup_daily_game_dataframe.T
matchup_daily_game_dataframe.columns = DAILY_COLUMNS
matchup_daily_game_dataframe

,Total,Mon,Tue,Wed,Thu,Fri,Sat,Sun
day,51,10,6,10,2,10,4,9


In [ ]:
DAILY_COLUMNS = ["Total", "Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]
def create_daily_game_count_dataframe(game_count_list, team_dict):
    """

    @param game_count_list:
    @return:
    """
    matchup_daily_game_dataframe = pd.DataFrame()
    matchup_daily_game_dataframe[team_dict[NAME_KEY]] = game_count_list
    matchup_daily_game_dataframe = matchup_daily_game_dataframe.T
    matchup_daily_game_dataframe.columns = DAILY_COLUMNS
    return matchup_daily_game_dataframe

In [4]:
DAILY_COLUMNS = ["Total", "Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]

def get_team_name_list_from_player_name_list(player_name_list):
    """

    @param player_name_list:
    @return:
    """
    team_name_list = list()
    for player in player_name_list:
        team = af.yahoo_player_team_and_jersey(player)[0]
        team_name_list.append(team)
    return player_name_list, team_name_list

def daily_get_player_games(team_game_counts_series, player_name_list, team_name_list):
    """

    @param team_game_counts_series:
    @param player_name_list:
    @param team_name_list:
    @return:
    """
    player_games_dictionary = dict()
    team_game_counts_dataframe = pd.DataFrame(team_game_counts_series)
    team_game_counts_dataframe.reset_index(inplace=True)
    team_game_counts_dataframe = team_game_counts_dataframe.replace(NBA_NAME_LA_CLIPPERS,
                                                                    YAHOO_NAME_LA_CLIPPERS)
    for player, team in zip(player_name_list,team_name_list):
        try:
            player_games_dictionary[player] = \
            team_game_counts_dataframe[team_game_counts_dataframe["index"] == team][0].iloc[0]
        except IndexError:
            player_games_dictionary[player] = 0
    return player_games_dictionary

def get_daily_game_count_list(filtered_games_week, player_name_list, team_name_list):

    daily_game_count_list = list()
    for day in list(filtered_games_week["gdte"].unique()):
        daily_dataframe = filtered_games_week[filtered_games_week["gdte"] == day]
        daily_team_game_counts = daily_dataframe[HOME_KEY].append(daily_dataframe[VISITORS_KEY])\
            .value_counts()
        player_games_dictionary = daily_get_player_games(daily_team_game_counts, player_name_list,
                                                         team_name_list)
        daily_game_count_list.append(sum(list(player_games_dictionary.values())))
    cleaned_daily_game_count_list = [10 if i >= 10 else i for i in daily_game_count_list]
    return daily_game_count_list, cleaned_daily_game_count_list


def get_matchup_days_as_column_names(filtered_games_week):
    """

    @param filtered_games_week:
    @return:
    """
    matchup_column_names = list()
    matchup_column_names.append("Total")
    matchup_column_names.append("Playable")
    for date in list(filtered_games_week[GDATE_KEY].unique()):
        date_format = pd.to_datetime(str(date))
        matchup_column_names.append(date_format.strftime("%a"))
    return matchup_column_names


def create_daily_game_count_dataframe(game_count_list, team_dict, matchup_column_names):
        """

        @param game_count_list:
        @return:
        """
        matchup_daily_game_dataframe = pd.DataFrame()
        matchup_daily_game_dataframe[team_dict[NAME_KEY]] = game_count_list
        matchup_daily_game_dataframe = matchup_daily_game_dataframe.T
        matchup_daily_game_dataframe.columns = matchup_column_names
        return matchup_daily_game_dataframe

def get_team_daily_game_count_dataframe_pipeline(team_dict, week="current"):
    """

    @param season_games_dataframe:
    @param week:
    @return:
    """
    season_games_dataframe = get_game_information_in_season(2020)
    if week == "next":
        fantasy_week, week_start_date, week_end_date = get_next_week_information()
    elif week == "current":
        fantasy_week, week_start_date, week_end_date = get_week_current_week_information()
    else:
        raise ValueError("Parameter 'week' must be 'current' or 'next'.")
    filtered_games_week, team_game_counts = get_fantasy_week_games_dataframe\
        (season_games_dataframe, week_start_date, week_end_date)
    player_id_name_team_list, player_name_list = get_player_ids_names_in_fantasy_team(team_dict)
    player_name_list, team_name_list = get_team_name_list_from_player_name_list(player_name_list)
    daily_game_count_list, cleaned_daily_game_count_list = get_daily_game_count_list\
        (filtered_games_week, player_name_list, team_name_list)
    total_and_daily_games_list = [sum(daily_game_count_list)] + [sum(cleaned_daily_game_count_list)]\
                                 + cleaned_daily_game_count_list
    matchup_column_names = get_matchup_days_as_column_names(filtered_games_week)
    team_daily_game_dataframe = create_daily_game_count_dataframe(total_and_daily_games_list,
                                                                  team_dict, matchup_column_names)
    return team_daily_game_dataframe

def get_matchup_daily_game_count_dataframe_pipeline(team1_dict, team2_dict, week="current"):
    """

    @param team1_dict:
    @param team2_dict:
    @param week:
    @return:
    """
    team1_daily_game_dataframe = get_team_daily_game_count_dataframe_pipeline(team1_dict, week)
    team2_daily_game_dataframe = get_team_daily_game_count_dataframe_pipeline(team2_dict, week)
    matchup_daily_game_count_dataframe = team1_daily_game_dataframe.append\
        (team2_daily_game_dataframe)
    return matchup_daily_game_count_dataframe

In [32]:
nunn = get_team_daily_game_count_dataframe_pipeline(af.NUNN_OF_YALL_BETTA, week="current")
ladron = get_team_daily_game_count_dataframe_pipeline(af.EL_LADRON_DE_CABRAS, week="current")

[2021-02-22 23:55:08,721 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-02-22 23:55:08,723 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 1446.0294632911682
[2021-02-22 23:55:08,723 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-02-22 23:55:10,772 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-02-22 23:55:10,774 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 1448.0804178714752
[2021-02-22 23:55:10,774 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-02-22 23:55:11,751 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-02-22 23:55:11,754 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 1449.0601267814636
[2021-02-22 23:55:11,754 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-02-22 23:55:12,645 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-02-22 23:55:12,647 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 1449.9535446166992
[2021-02-22 23:55:12,648 DEBUG] [yahoo_oauth.oau

In [35]:
matchup_daily_game_count_dataframe = nunn.append(ladron)

In [38]:
def get_matchup_daily_game_count_dataframe_pipeline(team1_dict, team2_dict, week="current"):
    """

    @param team1_dict:
    @param team2_dict:
    @param week:
    @return:
    """
    team1_daily_game_dataframe = get_team_daily_game_count_dataframe_pipeline(team1_dict, week)
    team2_daily_game_dataframe = get_team_daily_game_count_dataframe_pipeline(team2_dict, week)
    matchup_daily_game_count_dataframe = team1_daily_game_dataframe.append\
        (team2_daily_game_dataframe)
    return matchup_daily_game_count_dataframe

In [39]:
get_matchup_daily_game_count_dataframe_pipeline(af.NUNN_OF_YALL_BETTA, af.EL_LADRON_DE_CABRAS,
                                                week="current")

[2021-02-22 23:59:49,859 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-02-22 23:59:49,860 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 1727.1664657592773
[2021-02-22 23:59:49,861 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-02-22 23:59:51,650 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-02-22 23:59:51,655 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 1728.9608566761017
[2021-02-22 23:59:51,656 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-02-22 23:59:52,613 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-02-22 23:59:52,617 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 1729.9226183891296
[2021-02-22 23:59:52,617 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-02-22 23:59:53,506 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-02-22 23:59:53,516 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 1730.8216195106506
[2021-02-22 23:59:53,517 DEBUG] [yahoo_oauth.oau

,Total,Mon,Tue,Wed,Thu,Fri,Sat,Sun
Nunn of Y'all Betta,51,10,6,10,2,10,4,9
El Ladrón de Cabras,53,4,10,5,10,8,6,10


In [10]:
from datetime import date

today = date.today()
today.strftime("%a")

'Mon'

In [2]:

sc = af.yahoo_fantasy_api_authentication()
league = af.yahoo_fantasy_league(sc)

[2021-02-24 22:21:27,006 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-02-24 22:21:27,008 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 1233.283728837967
[2021-02-24 22:21:27,009 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID


In [4]:
league.player_details("Royce")

[{'player_key': '402.p.5905',
  'player_id': '5905',
  'name': {'full': "Royce O'Neale",
   'first': 'Royce',
   'last': "O'Neale",
   'ascii_first': 'Royce',
   'ascii_last': "O'Neale"},
  'editorial_player_key': 'nba.p.5905',
  'editorial_team_key': 'nba.t.26',
  'editorial_team_full_name': 'Utah Jazz',
  'editorial_team_abbr': 'Uta',
  'uniform_number': '23',
  'display_position': 'SF,PF',
  'headshot': {'url': 'https://s.yimg.com/iu/api/res/1.2/ylkW8KGAlVjTt8ZjYPIx_A--~C/YXBwaWQ9eXNwb3J0cztjaD0yMzM2O2NyPTE7Y3c9MTc5MDtkeD04NTc7ZHk9MDtmaT11bGNyb3A7aD02MDtxPTEwMDt3PTQ2/https://s.yimg.com/xe/i/us/sp/v/nba_cutout/players_l/01082021/5905.png',
   'size': 'small'},
  'image_url': 'https://s.yimg.com/iu/api/res/1.2/ylkW8KGAlVjTt8ZjYPIx_A--~C/YXBwaWQ9eXNwb3J0cztjaD0yMzM2O2NyPTE7Y3c9MTc5MDtkeD04NTc7ZHk9MDtmaT11bGNyb3A7aD02MDtxPTEwMDt3PTQ2/https://s.yimg.com/xe/i/us/sp/v/nba_cutout/players_l/01082021/5905.png',
  'is_undroppable': '0',
  'position_type': 'P',
  'primary_position': 'SF',
  'el

In [6]:
season_games_dataframe = get_game_information_in_season(2020)
fantasy_week, week_start_date, week_end_date = get_week_current_week_information()
filtered_games_week, team_game_counts = get_fantasy_week_games_dataframe\
    (season_games_dataframe, week_start_date, week_end_date)

[2021-03-01 19:40:45,922 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-03-01 19:40:45,924 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 657.6024804115295
[2021-03-01 19:40:45,925 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID


In [28]:
filtered_games_week


,gdte,gid,home,home_tid,visitors,visitors_tid
0,2021-03-01,0022000529,Orlando Magic,1.610613e+09,Dallas Mavericks,1.610613e+09
1,2021-03-01,0022000530,Philadelphia 76ers,1.610613e+09,Indiana Pacers,1.610613e+09
2,2021-03-01,0022000531,Chicago Bulls,1.610613e+09,Denver Nuggets,1.610613e+09
3,2021-03-01,0022000533,New Orleans Pelicans,1.610613e+09,Utah Jazz,1.610613e+09
4,2021-03-01,0022000534,San Antonio Spurs,1.610613e+09,Brooklyn Nets,1.610613e+09
...,...,...,...,...,...,...
65,2021-03-14,0022000597,Atlanta Hawks,1.610613e+09,Cleveland Cavaliers,1.610613e+09
66,2021-03-14,0022000598,Houston Rockets,1.610613e+09,Boston Celtics,1.610613e+09
67,2021-03-14,0022000599,Minnesota Timberwolves,1.610613e+09,Portland Trail Blazers,1.610613e+09
68,2021-03-14,0022000600,Chicago Bulls,1.610613e+09,Toronto Raptors,1.610613e+09


In [44]:
get_team_daily_game_count_dataframe_pipeline(af.NUNN_OF_YALL_BETTA)

[2021-03-01 20:32:39,498 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-03-01 20:32:39,500 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 3771.1784315109253
[2021-03-01 20:32:39,501 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN HAS EXPIRED
[2021-03-01 20:32:39,502 DEBUG] [yahoo_oauth.oauth.refresh_access_token] REFRESHING TOKEN
[2021-03-01 20:32:46,530 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-03-01 20:32:46,541 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 7.037970542907715
[2021-03-01 20:32:46,542 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-03-01 20:32:47,567 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-03-01 20:32:47,570 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 8.06714129447937
[2021-03-01 20:32:47,571 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-03-01 20:32:48,475 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-03-01 20:32:48,477 DEBUG] [yahoo_oauth.oauth.token_is_valid

Utah Jazz                 1
Brooklyn Nets             1
Philadelphia 76ers        1
Chicago Bulls             1
Houston Rockets           1
Charlotte Hornets         1
Orlando Magic             1
Portland Trail Blazers    1
Cleveland Cavaliers       1
New Orleans Pelicans      1
Denver Nuggets            1
Indiana Pacers            1
Dallas Mavericks          1
San Antonio Spurs         1
dtype: int64
Los Angeles Lakers    1
Miami Heat            1
Detroit Pistons       1
Boston Celtics        1
LA Clippers           1
San Antonio Spurs     1
New York Knicks       1
Phoenix Suns          1
Milwaukee Bucks       1
Toronto Raptors       1
Denver Nuggets        1
Washington Wizards    1
Memphis Grizzlies     1
Atlanta Hawks         1
dtype: int64
Los Angeles Lakers        1
Utah Jazz                 1
Dallas Mavericks          1
New Orleans Pelicans      1
Minnesota Timberwolves    1
Philadelphia 76ers        1
Golden State Warriors     1
Cleveland Cavaliers       1
Portland Trail Blazers

,Total,Playable,Mon,Tue,Wed,Thu,Sun,Wed,Thu,Fri,Sat,Sun
Nunn of Y'all Betta,72,69,7,10,6,10,0,1,10,8,9,8


In [38]:
nba_teams = set(list(season_games_dataframe["home"]) + list(season_games_dataframe["visitors"]))
len(nba_teams)

32

In [ ]:
def get_nba_team_daily_game_count_dataframe_pipeline(week="current"):
    """

    @param season_games_dataframe:
    @param week:
    @return:
    """
    season_games_dataframe = get_game_information_in_season(2020)
    if week == "next":
        fantasy_week, week_start_date, week_end_date = get_next_week_information()
    elif week == "current":
        fantasy_week, week_start_date, week_end_date = get_week_current_week_information()
    else:
        raise ValueError("Parameter 'week' must be 'current' or 'next'.")
    filtered_games_week, team_game_counts = get_fantasy_week_games_dataframe\
        (season_games_dataframe, week_start_date, week_end_date)
    for team in

    daily_game_count_list, cleaned_daily_game_count_list = get_daily_game_count_list\
        (filtered_games_week, player_name_list, team_name_list)
    total_and_daily_games_list = [sum(daily_game_count_list)] + [sum(cleaned_daily_game_count_list)]\
                                 + cleaned_daily_game_count_list
    matchup_column_names = get_matchup_days_as_column_names(filtered_games_week)
    team_daily_game_dataframe = create_daily_game_count_dataframe(total_and_daily_games_list,
                                                                  team_dict, matchup_column_names)
    return team_daily_game_dataframe

In [42]:
def get_nba_daily_game_count_list(filtered_games_week):

    daily_game_count_list = list()
    for day in list(filtered_games_week["gdte"].unique()):
        daily_dataframe = filtered_games_week[filtered_games_week["gdte"] == day]
        daily_team_game_counts = daily_dataframe[HOME_KEY].append(daily_dataframe[VISITORS_KEY])\
            .value_counts()

,gdte,gid,home,home_tid,visitors,visitors_tid
0,2021-03-01,0022000529,Orlando Magic,1.610613e+09,Dallas Mavericks,1.610613e+09
1,2021-03-01,0022000530,Philadelphia 76ers,1.610613e+09,Indiana Pacers,1.610613e+09
2,2021-03-01,0022000531,Chicago Bulls,1.610613e+09,Denver Nuggets,1.610613e+09
3,2021-03-01,0022000533,New Orleans Pelicans,1.610613e+09,Utah Jazz,1.610613e+09
4,2021-03-01,0022000534,San Antonio Spurs,1.610613e+09,Brooklyn Nets,1.610613e+09
...,...,...,...,...,...,...
65,2021-03-14,0022000597,Atlanta Hawks,1.610613e+09,Cleveland Cavaliers,1.610613e+09
66,2021-03-14,0022000598,Houston Rockets,1.610613e+09,Boston Celtics,1.610613e+09
67,2021-03-14,0022000599,Minnesota Timberwolves,1.610613e+09,Portland Trail Blazers,1.610613e+09
68,2021-03-14,0022000600,Chicago Bulls,1.610613e+09,Toronto Raptors,1.610613e+09


In [106]:
def move_column_inplace(df, col, pos):
    col = df.pop(col)
    df.insert(pos, col.name, col)

In [113]:
team_weekly_games_dataframe = pd.DataFrame()
for day in list(filtered_games_week["gdte"].unique()):
    date_format = pd.to_datetime(str(day))
    date_format = date_format.strftime("%a")
    daily_dataframe = filtered_games_week[filtered_games_week["gdte"] == day]
    daily_team_game_counts = daily_dataframe[HOME_KEY].append(daily_dataframe[VISITORS_KEY])\
        .value_counts()
    daily_team_game_counts_dataframe = pd.DataFrame(daily_team_game_counts, columns=[date_format])
    team_weekly_games_dataframe = pd.concat([team_weekly_games_dataframe,
                                             daily_team_game_counts_dataframe], axis=1)
team_weekly_games_dataframe = team_weekly_games_dataframe.fillna(0)
team_weekly_games_dataframe.drop("Team Durant", inplace=True)
team_weekly_games_dataframe.drop("Team LeBron", inplace=True)
team_weekly_games_dataframe.astype(int)
team_weekly_games_dataframe["Total"] = team_weekly_games_dataframe.sum(axis=1)
team_weekly_games_dataframe.sort_index()
move_column_inplace(team_weekly_games_dataframe, "Total", 0)

In [114]:
def get_nba_team_playing_games_in_week(filtered_games_week):
    """
    Function finds, for each day in the week, which NBA teams play a game that day.
    @param filtered_games_week:
    @return:
    """
    team_weekly_games_dataframe = pd.DataFrame()
    for day in list(filtered_games_week["gdte"].unique()):
        date_format = pd.to_datetime(str(day))
        date_format = date_format.strftime("%a")
        daily_dataframe = filtered_games_week[filtered_games_week["gdte"] == day]
        daily_team_game_counts = daily_dataframe[HOME_KEY].append(daily_dataframe[VISITORS_KEY])\
            .value_counts()
        daily_team_game_counts_dataframe = pd.DataFrame(daily_team_game_counts, columns=[date_format])
        team_weekly_games_dataframe = pd.concat([team_weekly_games_dataframe,
                                                 daily_team_game_counts_dataframe], axis=1)
    team_weekly_games_dataframe = team_weekly_games_dataframe.fillna(0)
    team_weekly_games_dataframe.drop("Team Durant", inplace=True)
    team_weekly_games_dataframe.drop("Team LeBron", inplace=True)
    team_weekly_games_dataframe.astype(int)
    team_weekly_games_dataframe["Total"] = team_weekly_games_dataframe.sum(axis=1)
    team_weekly_games_dataframe.sort_index()
    move_column_inplace(team_weekly_games_dataframe, "Total", 0)
    return team_weekly_games_dataframe

In [ ]:
def get_nba_team_playing_games_in_week(filtered_games_week):
    """
    Function finds, for each day in the week, which NBA teams play a game that day.
    @param filtered_games_week:
    @return:
    """
    team_weekly_games_dataframe = pd.DataFrame()
    for day in list(filtered_games_week["gdte"].unique()):
        date_format = pd.to_datetime(str(day))
        date_format = date_format.strftime("%a")
        daily_dataframe = filtered_games_week[filtered_games_week["gdte"] == day]
        daily_team_game_counts = daily_dataframe[HOME_KEY].append(daily_dataframe[VISITORS_KEY])\
            .value_counts()
        daily_team_game_counts_dataframe = pd.DataFrame(daily_team_game_counts, columns=[date_format])
        team_weekly_games_dataframe = pd.concat([team_weekly_games_dataframe,
                                                 daily_team_game_counts_dataframe], axis=1)
    team_weekly_games_dataframe = team_weekly_games_dataframe.fillna(0)
    team_weekly_games_dataframe.drop("Team Durant", inplace=True)
    team_weekly_games_dataframe.drop("Team LeBron", inplace=True)
    team_weekly_games_dataframe.astype(int)
    team_weekly_games_dataframe["Total"] = team_weekly_games_dataframe.sum(axis=1)
    team_weekly_games_dataframe.sort_index()
    move_column_inplace(team_weekly_games_dataframe, "Total", 0)
    return team_weekly_games_dataframe

def get_nba_team_playing_games_in_week_pipeline(week="current"):
    """

    @param week:
    @return:
    """
    season_games_dataframe = get_game_information_in_season(2020)
    if week == "next":
        fantasy_week, week_start_date, week_end_date = get_next_week_information()
    elif week == "current":
        fantasy_week, week_start_date, week_end_date = get_week_current_week_information()
    else:
        raise ValueError("Parameter 'week' must be 'current' or 'next'.")
    filtered_games_week, team_game_counts = get_fantasy_week_games_dataframe\
        (season_games_dataframe, week_start_date, week_end_date)
    team_weekly_games_dataframe = get_nba_team_playing_games_in_week(filtered_games_week)
    return team_weekly_games_dataframe
